In [ ]:
%pip install pandas
%pip install scikit-learn
%pip install request
%pip install matplotlib
%pip install numpy
%pip install joblib

In [ ]:
import pandas as pd
import mlflow
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score


# create a dataframe with data celsius and fahrenheit
df = pd.DataFrame({'celsius': [-40, -10, 0, 8, 15, 22, 38],
                     'fahrenheit': [-40, 14, 32, 46, 59, 72, 100]})

# show info
df.info()


In [ ]:
# params for linear regression
params = {
    'fit_intercept': True,
    'positive': True,
    'copy_X': True,
    'n_jobs': 10
}

# create a linear regression model
model = LinearRegression(**params)

# prepare data
X = df[['celsius']].values.reshape(-1, 1)
Y = df[['fahrenheit']].values

# Split data
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.25, random_state=50)

# Train model
model.fit(x_train, y_train)

# Predict
y_pred = model.predict([[89]])

print(y_pred)


In [ ]:
# export model

import joblib

joblib.dump(model, '../streamlit/model/temp.pkl')

In [ ]:
mlflow.set_tracking_uri('http://localhost:8080')
mlflow.set_experiment('Temperature')

with mlflow.start_run():

    mlflow.log_input(mlflow.data.from_pandas(df))
    mlflow.log_param('fit_intercept', params['fit_intercept'])
    mlflow.log_param('positive', params['positive'])
    mlflow.log_param('copy_X', params['copy_X'])
    mlflow.log_param('n_jobs', params['n_jobs'])

    mlflow.log_metric('mean_squared_error', mean_squared_error(y_test, model.predict(x_test)))
    mlflow.log_metric('mean_absolute_error', mean_absolute_error(y_test, model.predict(x_test)))
    mlflow.log_metric('r2_score', r2_score(y_test, model.predict(x_test)))

    mlflow.sklearn.log_model(model, 'model')
    mlflow.log_artifact('../streamlit/model/temp.pkl')
    